In [2]:
import pandas as pd
data = pd.read_csv("C:/Users/jstep/Downloads/german_credit_data/german-data-numeric.txt", sep=",",header=None)
del data[0]

In [3]:
data

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
0,1,6,4,12,5,5,3,4,1,67,...,0,0,1,0,0,1,0,0,1,1.0
1,2,48,2,60,1,3,2,2,1,22,...,0,0,1,0,0,1,0,0,1,2.0
2,4,12,4,21,1,4,3,3,1,49,...,0,0,1,0,0,1,0,1,0,1.0
3,1,42,2,79,1,4,3,4,2,45,...,0,0,0,0,0,0,0,0,1,1.0
4,1,24,3,49,1,3,3,4,4,53,...,1,0,1,0,0,0,0,0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,12,2,17,1,4,2,4,1,31,...,0,0,1,0,0,1,0,1,0,1.0
996,1,30,2,39,1,3,1,4,2,40,...,0,1,1,0,0,1,0,0,0,1.0
997,4,12,2,8,1,5,3,4,3,38,...,0,0,1,0,0,1,0,0,1,1.0
998,1,45,2,18,1,3,3,4,4,23,...,0,0,1,0,0,0,0,0,1,2.0


In [8]:
data = pd.read_csv("C:/Users/jstep/Downloads/german_credit_data/german.data", sep=" ",header=None)
col_mapping = {0:"checking_account", 
               1:"duration",
               2:"credit_history",
               3:"purpose",
               4:"credit_amt",
               5:"savings_acct",
               6:"employment",
               7:"installment_rate_percent",
               8:"gender_status",
               9:"other_debtors",
              10:"residence_since",
              11:"property",
              12:"age",
              13:"other_installment_plans",
              14:"housing",
              15:"no_existing_credits",
              16:"job",
              17:"no_people_liable",
              18:"has_telephone",
              19:"is_foreign_worker",
              20:""}

In [10]:
data.columns = ["V"+str(col) for col in data.columns]

In [12]:
data = data.rename(columns={"V20":"predicted"})

In [13]:
data

,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V11,V12,V13,V14,V15,V16,V17,V18,V19,predicted
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,A14,12,A32,A42,1736,A61,A74,3,A92,A101,...,A121,31,A143,A152,1,A172,1,A191,A201,1
996,A11,30,A32,A41,3857,A61,A73,4,A91,A101,...,A122,40,A143,A152,1,A174,1,A192,A201,1
997,A14,12,A32,A43,804,A61,A75,4,A93,A101,...,A123,38,A143,A152,1,A173,1,A191,A201,1
998,A11,45,A32,A43,1845,A61,A73,4,A93,A101,...,A124,23,A143,A153,1,A173,1,A192,A201,2
